# 2. Update WIAG-IDs in FactGrid and then add FG-IDs in WIAG



The code is designed to synchronize identification numbers (IDs) between a local database (WIAG) and an online database called FactGrid. Here's a detailed explanation:



1. Load local data: The program reads a local spreadsheet that contains personal information and their corresponding IDs.

2. Retrieve online data: It connects to the FactGrid online database to fetch the current IDs associated with the same individuals.

3. Identify discrepancies: The code compares the local IDs with the online IDs to find any differences or mismatches.

4. Find entries to update in FactGrid: Check for FG-entries that have outdated WIAG-IDs, which need to be updated (and for which it can be done automatically)

5. Update FactGrid: A file listing the discrepancies is generated, formatted in a way that can be used to automatically update the IDs in the FactGrid database. This needs to be uploaded manually.

6. Retrieve updated online data: The data from FactGrid is redownloaded, now that changes were made.

7. Rerunning checks: To make sure that no mistakes have been introduced by updating FactGrid, the checks from before are run again.

8. Find entries to update in WIAG: Check for WIAG-entries that do not link to an FG-entry, but an FG-entry links to them and add the respective FG-ID to them

9. Update WIAG: WIAG-entries that do not yet link to a FactGrid-entry, but to which an FG-entry links, are updated to link back

In essence, the code helps maintain accurate and consistent records between the local files and the online database by identifying mismatched IDs and preparing the necessary updates.

## 1. Import local (WIAG) data



### Export Csv Personendaten



 - Go to https://wiag-vocab.adw-goe.de/query/can

 - Click on the Export button and then on CSV Personendaten

 - This will start downloading the data.

### Import the contents of the CSV file

Once the download has finished, move the file to the `input_path` path (defined in the cell below) or change `input_path` to where the file is located.

Lastly, if you renamed the file, change `input_file` to the actual name.

In [1]:
import requests
import csv
import os
import pandas as pd
import json

#change input_path if your file is located somewhere else, e.g. to "C:\Users\schwart2\Downloads""
input_path = r"C:\Users\Public\sync_notebooks\input_files"
#change input_file if you renamed the file
input_file = f"WIAG-Domherren-DB-Lebensdaten.csv"

input_path_file = os.path.join(input_path, input_file)
wiag_persons_df = pd.read_csv(input_path_file, sep=';')
wiag_persons_df = wiag_persons_df[['FactGrid_ID', 'id']] # selecting columns
print(str(len(wiag_persons_df)) + " entries were imported.")

wiag_persons_df.columns = ['wiag_fg_id', 'wiag_id'] # renaming the columns


17087 entries were imported.


## 2. Import Factgrid data

This downloads and imports the data from FactGrid automatically.

In [2]:
fg_url = 'https://database.factgrid.de/sparql'
fg_query = """
SELECT ?person ?wiag WHERE {
  ?person wdt:P601 ?wiag.
  ?person wdt:P2 wd:Q7.
}
"""

r = requests.get(fg_url, params={'query': fg_query}, headers={"Accept": "application/json"})
data = r.json()
fg_wiag_ids_df = pd.json_normalize(data['results']['bindings'])

print(str(len(fg_wiag_ids_df)) + " entries were imported.")

#drop irrelevant columns
fg_wiag_ids_df.drop(columns=[column for column in fg_wiag_ids_df.columns if column.endswith('type')], inplace=True)
fg_wiag_ids_df.drop(columns=[column for column in fg_wiag_ids_df.columns if column.endswith('xml:lang')], inplace=True)

#extract q ID
fg_wiag_ids_df['person.value'] = fg_wiag_ids_df['person.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

#set column names
fg_wiag_ids_df.columns = ['fg_id', 'fg_wiag_id']


14015 entries were imported.


## 3. Check for problematic entries

For this section: Should any entries be listed, these need to be **fixed manually** before starting again by exporting the data (step 1).



This checks whether any FactGrid-entries link to multiple WIAG-IDs and lists them.

In [3]:
fg_wiag_ids_df[fg_wiag_ids_df.duplicated(subset = ['fg_id'], keep = False)].sort_values(by = 'fg_id')

,fg_id,fg_wiag_id


This checks whether any WIAG-entries link to multiple FactGrid-IDs and lists them.

In [4]:
wiag_persons_df[wiag_persons_df.duplicated(subset = ['wiag_id'], keep = False)].sort_values(by = 'wiag_id')

,wiag_fg_id,wiag_id


This checks whether any FactGrid-entries link to the same WIAG-ID.

In [5]:
fg_wiag_ids_df[fg_wiag_ids_df.duplicated(subset = ['fg_wiag_id'], keep = False)].sort_values(by = ['fg_wiag_id'])

,fg_id,fg_wiag_id


For the listed IDs, a WIAG-entry links to a FactGrid-entry, which does not yet link to any WIAG-entry

In [6]:
#merge dataframes (outer join on WIAG-ID)
outer_df = fg_wiag_ids_df.merge(wiag_persons_df, how='outer' , left_on = 'fg_wiag_id', right_on = 'wiag_id')
fg_missing_wiag_id = outer_df[~outer_df['wiag_fg_id'].isna() & outer_df['fg_id'].isna()]

fg_missing_wiag_id[['wiag_id', 'wiag_fg_id']]

,wiag_id,wiag_fg_id


## 4. Find entries to update

This section finds entries to update by first checking all WIAG-IDs linked to from FG-entries (section A) and then (section B) checking FG-IDs that WIAG-entries link to.

### a) Check all WIAG-IDs that FactGrid-entries link to

This section finds entries to update by checking WIAG-IDs that are referenced in FactGrid.

Please note, that the code cell below can take **up to 10 minutes.** Should the cell fail, try running it again (likely cause is a timeout).

In [7]:
from scripts.fg_wiag_ids_functions import check_fg

#creates a list with the WIAG-ID and FactGrid-ID paired for each entry 
entries_to_be_checked = list(zip(list(fg_wiag_ids_df['fg_wiag_id']), list(fg_wiag_ids_df['fg_id'])))

#description of the outputs of the check_fg function:
#entries_to_be_updated: FactGrid-IDs which point to an outdated WIAG-ID (WIAG redirected to a newer one) and for which the new WIAG entry does not point to the FactGrid-ID
#wiag_different_fgID: WIAG-IDs that link to a different FG-ID from the one that points to them
#wiag_missing_fgID: WIAG-IDs to which a FactGrid-entry points, but which point to no FactGrid-ID

entries_to_be_updated, wiag_different_fgID, wiag_missing_fgID = await check_fg(entries_to_be_checked)


Starting attempt #1
3000/14015 checked. Missed count: 169
6000/14015 checked. Missed count: 434
9000/14015 checked. Missed count: 476
12000/14015 checked. Missed count: 922
14015/14015 checked. Missed count: 965
Finalized all. Couldn't get data for 965 entries
Starting attempt #2
965/965 checked. Missed count: 2
Finalized all. Couldn't get data for 2 entries
Starting attempt #3
2/2 checked. Missed count: 0
Finalized all. Finished fetching data for all entries.


If the cell below lists any entries, these entries **needs to be fixed manually**. After fixing entries, you need to **start again** from step 1!

The listed WIAG-entries link to FactGrid-entries, but the FG-entries don't link back, but instead link to a different WIAG-entry.

In [8]:
wiag_different_fgID


,fg_wiag_id,wiag_redirected,fg_id,wiag_fg_id


The following entries point to outdated WIAG-IDs and will be updated automatically. You should **check a sample** of the output and also make sure that the amount of entries isn't absurdly high.

Should there be no output, that means that no entries to be updated were found.

In [9]:
pd.DataFrame(entries_to_be_updated, columns = ["fg_id", "fg_wiag_id", "new_wiag_id"])


,fg_id,fg_wiag_id,new_wiag_id


### b) Check FactGrid-IDs that WIAG-entries point to

This checks for FactGrid-entries, where a WIAG entry is linking to them, but the FG-entry links to a different WIAG-entry.

The output **needs to be checked fully** (if there is any). The expected solution (which will be carried out automatically) is to update the FactGrid-entry with the listed WIAG-ID, however it's a good idea to check whether this makes sense for all entries.

In [10]:
#merge dataframes (inner join on FactGrid-ID)
merged_df = fg_wiag_ids_df.merge(wiag_persons_df, left_on='fg_id', right_on = 'wiag_fg_id')

#check for entries where the WIAG-ID in FactGrid is different from the one in WIAG
fg_diff_wiag_id = merged_df[merged_df['fg_wiag_id'] != merged_df['wiag_id']]
fg_diff_wiag_id = fg_diff_wiag_id[['fg_id', 'fg_wiag_id', 'wiag_id']] # selecting columns - no need to show the same FG-ID twice

fg_diff_wiag_id

,fg_id,fg_wiag_id,wiag_id


## 5. Update FactGrid

If no entries are listed by the cell below, you can skip all the way to step 8 (no updates need to be done -> no data needs to be redownloaded -> no checks need to run again). You should have checked both lists of updates before this step, if not, check the list now.

The `qid` is the FactGrid-ID for which an update should be performed. The `-P601` column shows the WIAG-ID which will be removed from the FactGrid-entry. The `P601` column shows the WIAG-ID which will be added to the entry.

In [11]:
#List the entries in a format that FactGrid understands. (used for updating FactGrid automatically)
fg_qs_csv = fg_diff_wiag_id
fg_qs_csv.columns = ['qid', '-P601', 'P601']
#fg_qs_csv = fg_qs_csv.set_index('qid')

#add the updates from the first half of step 4
final_fg_qs_csv = pd.concat([fg_qs_csv, entries_to_be_updated])# , ignore_index=True) TODO necessary?
final_fg_qs_csv # list some entries

,qid,-P601,P601


### Generate update-file

Once all (if any popped up) problematic entries have been taken care of, the file for updating FactGrid can be generated.



You can change `output_path` below to where you want the generated file to be saved to.

In [ ]:
output_path = r"C:\Users\Public\sync_notebooks\output_files" # change this to save the file to somewhere else

from datetime import datetime
today_string = datetime.now().strftime('%Y-%m-%d') # create a timestamp for the name of the output file

final_fg_qs_csv["-P601"] = final_fg_qs_csv["-P601"].apply(lambda x: f'"{x}"') # putting quotes around the value
final_fg_qs_csv["P601"] = final_fg_qs_csv["P601"].apply(lambda x: f'"{x}"') # putting quotes around the value
final_fg_qs_csv.to_csv( # generate csv file
    os.path.join(
        output_path,
        f'factgrid_wiag_id_update_{today_string}.csv'
    ),
    index=False
)


### Upload the file



The generated Factgrid file can be uploaded on to quick statements [here](https://database.factgrid.de/quickstatements/#/batch). More details to perform this [can be found here](https://github.com/WIAG-ADW-GOE/sync_notebooks/blob/main/docs/Run_factgrid_csv.md)



## 6. Retrieve updated online data

Now that FactGrid has been updated, the data has to be redownloaded. Consequently this is almost the same code as in step 2 (the url and query variables from above are also reused)

In [12]:
r = requests.get(fg_url, params={'query': fg_query}, headers={"Accept": "application/json"})
data = r.json()
fg_wiag_ids_df = pd.json_normalize(data['results']['bindings'])

print(str(len(fg_wiag_ids_df)) + " entries were imported.")

#drop irrelevant columns
fg_wiag_ids_df.drop(columns=[column for column in fg_wiag_ids_df.columns if column.endswith('type')], inplace=True)
fg_wiag_ids_df.drop(columns=[column for column in fg_wiag_ids_df.columns if column.endswith('xml:lang')], inplace=True)

#extract q ID
fg_wiag_ids_df['person.value'] = fg_wiag_ids_df['person.value'].map(lambda x: x.strip('https://database.factgrid.de/entity/'))

#set column names
fg_wiag_ids_df.columns = ['fg_id', 'fg_wiag_id']


14015 entries were imported.


## 7. Rerunning checks

To make sure that no mistakes have been introduced by updating FactGrid, the checks from before are run again.

This checks whether any FactGrid-entries link to multiple WIAG-IDs and lists them.

In [13]:
fg_wiag_ids_df[fg_wiag_ids_df.duplicated(subset = ['fg_id'], keep = False)].sort_values(by = 'fg_id')

,fg_id,fg_wiag_id


This checks whether any FactGrid-entries link to the same WIAG-ID.

In [14]:
fg_wiag_ids_df[fg_wiag_ids_df.duplicated(subset = ['fg_wiag_id'], keep = False)].sort_values(by = 'fg_wiag_id') 

,fg_id,fg_wiag_id


For the listed IDs, a WIAG-entry links to a FactGrid-entry, which does not yet link to any WIAG-entry

In [15]:
#merge dataframes (outer join on WIAG-ID) - reusing WIAG-dataframe from step 1
outer_df = fg_wiag_ids_df.merge(wiag_persons_df, how='outer' , left_on = 'fg_wiag_id', right_on='wiag_id')
outer_df = outer_df[~outer_df['wiag_fg_id'].isna() & outer_df['fg_id'].isna()]

outer_df[['wiag_id', 'wiag_fg_id']]

,wiag_id,wiag_fg_id


## 8. Find entries to update in WIAG

For the following list, the FactGrid-entry is linking to the WIAG-entry, but the WIAG-entry links to no the FG-entry. These entries will be updated automatically to link back. You should **check a sample** and make sure the number of updates is not absurdly high (greater than 500).

Should no entries be listed, this means that no update needs to be performed. In this case you should skip the rest of the notebook and go straight to [notebook 3](Csv2FactGrid-create.ipynb) (Csv2FactGrid-create).

In [16]:
#merge dataframes - reusing WIAG-dataframe from step 1
new_merged_df = fg_wiag_ids_df.merge(wiag_persons_df, left_on = 'fg_wiag_id', right_on='wiag_id')
new_merged_df = new_merged_df[~new_merged_df['wiag_id'].str.startswith('WIAG-Pers-EPISCGatz')] # don't update bishops

#find WIAG-entries which do not link to an FG-ID, but an FG-entry links to the WIAG-ID => update WIAG-entries with FG-ID
to_be_updated_df = new_merged_df[new_merged_df['wiag_fg_id'].isna() & ~new_merged_df['fg_id'].isna()]
to_be_updated_df = to_be_updated_df[['fg_id', 'wiag_id']]
to_be_updated_df

,fg_id,wiag_id
9694,Q1773191,WIAG-Pers-CANON-53051-001
9695,Q1773192,WIAG-Pers-CANON-53145-001
9696,Q1773193,WIAG-Pers-CANON-53740-001
9697,Q1773464,WIAG-Pers-CANON-47214-001
9698,Q1773465,WIAG-Pers-CANON-48805-001
...,...,...
10066,Q1773792,WIAG-Pers-CANON-49653-001
10067,Q1773793,WIAG-Pers-CANON-49654-001
10068,Q1773794,WIAG-Pers-CANON-49655-001
10069,Q1773795,WIAG-Pers-CANON-49657-001


Exporting the list to a CSV-file, so the entirety of proposed updates can be checked easily. Change the `output path` if you want the file to be saved to somewhere else.

In [17]:
output_path = r"C:\Users\Public\sync_notebooks\output_files"


In [18]:
from datetime import datetime

today_string = datetime.now().strftime('%Y-%m-%d')

to_be_updated_df.to_csv( # generate csv file
    os.path.join(
        output_path,
        f'wiag_ids_to_be_updated_{today_string}.csv'
    ),
    index=False
)

## 9. Update WIAG

### Generate SQL file

From the list above an SQL-file will be generated, which then needs to be uploaded. The same `output path` as above will be used.

In [19]:
from datetime import datetime

query = "LOCK TABLES url_external WRITE, item_corpus WRITE;\n"
for row in to_be_updated_df.itertuples():
    query += f"""
INSERT INTO url_external (item_id, value, authority_id)
SELECT item_id, '{row.fg_id}', 42 FROM item_corpus
WHERE id_public = "{row.wiag_id}";
"""
query += "\nUNLOCK TABLES;"

today_string = datetime.now().strftime('%Y-%m-%d')
with open(os.path.join(output_path, f'insert-uext-can_{today_string}.sql'), 'w') as file:
    file.write(query)

### Upload file

Now that the file has been generated, you need to upload the file to the WIAG database. As always go to [phpMyAdmin (WIAG)](https://vwebfile.gwdg.de/phpmyadmin/), then first select the database (wiagvokabulare) and then either go to the `Import` tab and choose the file to run or paste the contents of the SQL-file into the textfield (more details here [Run_SQL_Query_and_Export_CSV.md](https://github.com/WIAG-ADW-GOE/sync_notebooks/blob/main/docs/Run_SQL_Query_and_Export_CSV.md)).

### Next notebook

Once the update is done, you can continue with [notebook 3](Csv2FactGrid-create.ipynb) (Csv2FactGrid-create).